In [1]:
from __future__ import print_function

import os
import gzip
import wget
import keras
import numpy as np
import ftplib
import pathlib

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from sklearn.model_selection import train_test_split


Using TensorFlow backend.


Load up a full set

In [3]:
# the set becomes pretty massive due to the amount of memory that integer lists eat
ncbi = "ftp.ncbi.nlm.nih.gov"
src = "/refseq/release/bacteria/"

X = [] # values
Y = [] # lables

# turning the sequences into sort-of normalized number lists
base_map = {
    "A": [1, 0, 0], 
    "T": [1, 0, 1], 
    "C": [1, 1, 0], 
    "G": [1, 1, 1], 
    "N": [0, 0, 0] # the N is used for padding? 
} 
lable_map = {}

w_size = 108 # will just match up the read length as the "image size" (needs to be a number that can be rooted)
#w_size = 100
s_size = 200 # the step size (how many bases should it move forward for the next "image")

# pseudo image width and height
size = int((w_size*3)**(1/2))
#size = int((w_size)**(1/2))

ftp = ftplib.FTP()
ftp.connect(ncbi)
ftp.login()

# need to make sure that the dirs that we're downloading stuff to exist
pathlib.Path(os.getcwd()+src).mkdir(parents=True, exist_ok=True) 

# work through all of the bacteria genome files
for f in ftp.nlst(src):
    if ".genomic.fna.gz" in f:
        path = os.getcwd()+f # making an absolute path
        
        if os.path.isfile(path):
            print(f+" already present")
        else:
            print("Fetching "+f)
            # couldn't figure out how to pull files via the ftp library - using wget for this
            wget.download("ftp://"+ncbi+f, out=path)

        fgz = gzip.open(path, "rb")
        file_content = fgz.read().decode("utf-8")

        curr_k = ""
        curr_v = ""
        for l in file_content.split("\n"):
            if ">" in l: # find lines that contain sequence titles
                if curr_v != "": # skipping the empty sequences (mainly to skip the first loop)
                    if curr_k not in lable_map:
                        lable_map[curr_k] = len(lable_map)
                    # turning the sequence into overlapping n base chunks
                    for i in range(0, (len(curr_v)-w_size)//s_size):
                        # turning the mask into a binary list
                        tmp = sum([base_map[j] for j in curr_v[i*s_size:i*s_size+w_size]], []) 
                        # turning the list into a fake image and dropping it into the set
                        X.append([tmp[k*size:k*size+size] for k in range(size)])
                        Y.append(lable_map[curr_k])
                curr_k = l
                curr_v = ""
            else: # gathering up the full sequence
                curr_v += l 
                
        break

X = np.array(X)
Y = np.array(Y)
        

/refseq/release/bacteria/bacteria.1779.1.genomic.fna.gz already present


Only load a partial set

In [5]:
# the set becomes pretty massive due to the amount of memory that integer lists eat
# only allow unique subsequences per sample
ncbi = "ftp.ncbi.nlm.nih.gov"
src = "/refseq/release/bacteria/"

XY = {} # lable + value tuples

# turning the sequences into sort-of normalized number lists
base_map = {
    "A": [1, 0, 0], 
    "T": [1, 0, 1], 
    "C": [1, 1, 0], 
    "G": [1, 1, 1], 
    "N": [0, 0, 0] # the N is used for padding? 
} 
lable_map = {}

w_size = 108 # will just match up the read length as the "image size" (needs to be a number that can be rooted)
#w_size = 100
s_size = 200 # the step size (how many bases should it move forward for the next "image")

# pseudo image width and height
size = int((w_size*3)**(1/2))
#size = int((w_size)**(1/2))

ftp = ftplib.FTP()
ftp.connect(ncbi)
ftp.login()

# need to make sure that the dirs that we're downloading stuff to exist
pathlib.Path(os.getcwd()+src).mkdir(parents=True, exist_ok=True) 

# work through all of the bacteria genome files
for f in ftp.nlst(src):
    if ".genomic.fna.gz" in f:
        path = os.getcwd()+f # making an absolute path
        
        if os.path.isfile(path):
            print(f+" already present")
        else:
            print("Fetching "+f)
            # couldn't figure out how to pull files via the ftp library - using wget for this
            wget.download("ftp://"+ncbi+f, out=path)

        fgz = gzip.open(path, "rb")
        file_content = fgz.read().decode("utf-8")

        curr_k = ""
        curr_v = ""
        for l in file_content.split("\n"):
            if ">" in l: # find lines that contain sequence titles
                if curr_v != "": # skipping the empty sequences (mainly to skip the first loop)
                    if curr_k not in lable_map:
                        lable_map[curr_k] = len(lable_map)
                    # turning the sequence into overlapping n base chunks
                    for i in range(0, (len(curr_v)-w_size)//s_size):
                        # turning the list into a fake image and dropping it into the set
                        tmp_X = curr_v[i*s_size:i*s_size+w_size]
                        tmp_Y = lable_map[curr_k]
                        if (tmp_X, tmp_Y) not in XY: # only add it if 
                            XY[(tmp_X, tmp_Y)] = True
                curr_k = l
                curr_v = ""
            else: # gathering up the full sequence
                curr_v += l 
                
        break

X = []
Y = []
      
for k in XY.keys():
    # turning the mask into a binary list
    tmp = sum([base_map[j] for j in k[0]], []) 
    X.append([tmp[j*size:j*size+size] for j in range(size)])
    Y.append(k[1])
    
X = np.array(X)
Y = np.array(Y)
        

/refseq/release/bacteria/bacteria.1779.1.genomic.fna.gz already present


Train the model

In [ ]:
batch_size = 128
num_classes = len(lable_map)
epochs = 12

# split the sets into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

# reshaping the data so that it fits in the cnn demo code
X_train = X_train.reshape(X_train.shape[0], size, size, 1)
X_test = X_test.reshape(X_test.shape[0], size, size, 1)
input_shape = (size, size, 1)

# convert class vectors to binary class matrices
Y_train = keras.utils.to_categorical(Y_train, num_classes)
Y_test = keras.utils.to_categorical(Y_test, num_classes)

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'X_train samples')
print(X_test.shape[0], 'X_test samples')

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


X_train shape: (1440708, 18, 18, 1)
1440708 X_train samples
709603 X_test samples
Train on 1440708 samples, validate on 709603 samples
Epoch 1/12
1440708/1440708 [==============================] - 550s 382us/step - loss: 4.9308 - accuracy: 0.0175 - val_loss: 4.8656 - val_accuracy: 0.0178
Epoch 2/12
1440708/1440708 [==============================] - 549s 381us/step - loss: 4.8837 - accuracy: 0.0183 - val_loss: 4.8453 - val_accuracy: 0.0185
Epoch 3/12
1440708/1440708 [==============================] - 478s 332us/step - loss: 4.8751 - accuracy: 0.0183 - val_loss: 4.8403 - val_accuracy: 0.0182
Epoch 4/12
1440708/1440708 [==============================] - 473s 328us/step - loss: 4.8676 - accuracy: 0.0181 - val_loss: 4.8176 - val_accuracy: 0.0186
Epoch 5/12
1440708/1440708 [==============================] - 460s 320us/step - loss: 4.8625 - accuracy: 0.0181 - val_loss: 4.8164 - val_accuracy: 0.0189
Epoch 6/12
1440708/1440708 [==============================] - 476s 330us/step - loss: 4.8597 - 